### Problem Statement
Imagine you are a highly-indemand actor, who has been presented with offers to star in n different movie projects under development. Each offer comes specified with the first and last  day of filming. To take the job, you must commit to being available throughout  this entire period. Thus you cannot simultaneously accept two jobs whose  intervals overlap.

### Criteria
You want to make as much money as possible. Because each of these  films pays the same fee per film, this implies you seek the largest possible  set of jobs (intervals) such that no two of them conflict with each other.

In [1]:
import numpy as np
import random
import datetime
from collections import namedtuple
import calendar

np.random.seed(314)
Range = namedtuple("Range", ["start", "end"])


### Creating a DateRange Class

In [20]:
class DateRange(object):
    def __init__(self, date_range):
        self.range = date_range
        self.difference = self.range.end-self.range.start
        
    def get_start_date(self):
        return self.range.start
    
    def get_end_date(self):
        return self.range.end
    
    def get_start_string(self):
        return self.range.start.strftime("%Y-%m-%d")
    
    def get_end_string(self):
        return self.range.end.strftime("%Y-%m-%d")
    
    def __str__(self):
        return f"{self.get_start_string()} ====> {self.get_end_string()}"
        
    def __lt__(self, r2):
        return self.range < r2
    
    def __le__(self, r2):
        return self.range <= r2
    
    def __eq__(self, r2):
        return self.range == r2
    
    def __gt__(self, r2):
        return self.range > r2
    
    def __ge__(self, r2):
        return self.range >= r2
    
    def get_overlap(self, r2):
        latest_start = max(self.range.start, r2.range.start)
        earliest_end = min(self.range.end, r2.range.end)
        delta = (earliest_end - latest_start).days + 1
        return max(0, delta)
    
    def is_overlap(self, r2):
        return self.get_overlap(r2) != 0


### Creating a Test Set

In [38]:
def make_datetime_object(day,month,year):
    return datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")
    
def get_random_date(start):
    """
    Generates a random datetime object between two other datetime objects.
    """
    MAX_DAYS = 30
    MIN_DAYS = 3
    number_of_days = np.random.randint(MIN_DAYS, MAX_DAYS)
    return start + datetime.timedelta(days=number_of_days)
    
def get_date(spread, year):
    month = np.random.randint(1,spread)
    day = np.random.randint(1,calendar.monthrange(year,month)[1])

    return make_datetime_object(day,month,year)

def generate_jobs(n,MAX_DAYS,MIN_DAYS,spread,year):
    date_ranges=[]
    for i in range(n):
        start = get_date(spread, year)
        end = get_random_date(start)
        date_range = DateRange(Range(start=start, end=end))
        date_ranges.append(date_range)
    return date_ranges

def check_overlap(date, date_ranges):
    _temp = []
    for d in date_ranges:
        print(f"Checking overlap with previous date: {d}")
        _temp.append(date.is_overlap(d))
    if not any(_temp):
        print("No overlap with any previous date.")
        return True
    return False

### Generate a sample test set

In [39]:
date_ranges = generate_jobs(10,3,30,3,2019)

### Earliest Job First Methodology
The simplest idea to solve this scheduling problem is to accept the work which starts the earliest and then go down the list of jobs and take the work which is next to start and doesnt overlap with any previously taken assignments.

In [40]:
def earliest_job_first(date_ranges):
    dates_to_take = []
    sorted_date_ranges = np.argsort(date_ranges)
    dates_to_take.append(date_ranges[sorted_date_ranges[0]])
    print(f"Job which starts the first: {date_ranges[sorted_date_ranges[0]]}")
    start = 0
    for other_date in sorted_date_ranges[1:]:
        print(f"Next earliest starting job: {date_ranges[other_date]}")
        take_date = check_overlap(date_ranges[other_date], dates_to_take)
        if take_date:
            dates_to_take.append(date_ranges[other_date])
            print(f"Appending to taken jobs list: {date_ranges[other_date]}")

        print("===================================================")
    return dates_to_take
            
jobs_to_take = earliest_job_first(date_ranges)

Job which starts the first: 2019-01-06 ====> 2019-01-16
Next earliest starting job: 2019-01-17 ====> 2019-01-21
Checking overlap with previous date: 2019-01-06 ====> 2019-01-16
No overlap with any previous date.
Appending to taken jobs list: 2019-01-17 ====> 2019-01-21
Next earliest starting job: 2019-01-17 ====> 2019-02-03
Checking overlap with previous date: 2019-01-06 ====> 2019-01-16
Checking overlap with previous date: 2019-01-17 ====> 2019-01-21
Next earliest starting job: 2019-02-07 ====> 2019-03-03
Checking overlap with previous date: 2019-01-06 ====> 2019-01-16
Checking overlap with previous date: 2019-01-17 ====> 2019-01-21
No overlap with any previous date.
Appending to taken jobs list: 2019-02-07 ====> 2019-03-03
Next earliest starting job: 2019-02-12 ====> 2019-02-23
Checking overlap with previous date: 2019-01-06 ====> 2019-01-16
Checking overlap with previous date: 2019-01-17 ====> 2019-01-21
Checking overlap with previous date: 2019-02-07 ====> 2019-03-03
Next earliest 

In [41]:
print("Checking jobs and pay:")
for job in jobs_to_take:
    print(job)
    print("Pay: $X")
print(f"=============================================")

print(f"Total Pay: ${len(jobs_to_take)}X")
print(f"Total Jobs Taken: {len(jobs_to_take)}")
print(f"Total Percent of Jobs Taken: {(len(jobs_to_take)/len(date_ranges))*100}%")

Checking jobs and pay:
2019-01-06 ====> 2019-01-16
Pay: $X
2019-01-17 ====> 2019-01-21
Pay: $X
2019-02-07 ====> 2019-03-03
Pay: $X
Total Pay: $3X
Total Jobs Taken: 3
Total Percent of Jobs Taken: 30.0%


This idea makes sense until we realize that accepting the earliest job might block us from taking many other jobs if that first job is quite long. For example if the first movie you accept takes 30 days to film and there are 3 overlapping movie which only take 10 days each to film, then you are accepting 1 movie at the cost of 3.

This brings us to another approach:

### Shortest Job First Methodology
The idea here is that we accept the movie which finishes in the shortest time and then go down the list accepting the movies which take the next shortest time and dont clash with previously accepted movies.

In [42]:
def shortest_movie_first(date_ranges):
    differences = np.argsort([i.difference.days for i in date_ranges])
    print(f"Movie with shortest duration: {date_ranges[differences[0]]}")
    dates_to_take = [date_ranges[differences[0]]]
    for other_date in differences[1:]:
        print(f"Next earliest starting job: {date_ranges[other_date]}")
        take_date = check_overlap(date_ranges[other_date], dates_to_take)
        
        if take_date:
            dates_to_take.append(date_ranges[other_date])
            print(f"Appending to taken jobs list: {date_ranges[other_date]}")

        print("===================================================")
    return dates_to_take

jobs_to_take = shortest_movie_first(date_ranges)

Movie with shortest duration: 2019-02-20 ====> 2019-02-24
Next earliest starting job: 2019-01-17 ====> 2019-01-21
Checking overlap with previous date: 2019-02-20 ====> 2019-02-24
No overlap with any previous date.
Appending to taken jobs list: 2019-01-17 ====> 2019-01-21
Next earliest starting job: 2019-02-17 ====> 2019-02-23
Checking overlap with previous date: 2019-02-20 ====> 2019-02-24
Checking overlap with previous date: 2019-01-17 ====> 2019-01-21
Next earliest starting job: 2019-01-06 ====> 2019-01-16
Checking overlap with previous date: 2019-02-20 ====> 2019-02-24
Checking overlap with previous date: 2019-01-17 ====> 2019-01-21
No overlap with any previous date.
Appending to taken jobs list: 2019-01-06 ====> 2019-01-16
Next earliest starting job: 2019-02-12 ====> 2019-02-23
Checking overlap with previous date: 2019-02-20 ====> 2019-02-24
Checking overlap with previous date: 2019-01-17 ====> 2019-01-21
Checking overlap with previous date: 2019-01-06 ====> 2019-01-16
Next earlies

In [43]:
print("Checking jobs and pay:")
for job in jobs_to_take:
    print(job)
    print("Pay: $X")
print(f"=============================================")

print(f"Total Pay: ${len(jobs_to_take)}X")
print(f"Total Jobs Taken: {len(jobs_to_take)}")
print(f"Total Percent of Jobs Taken: {(len(jobs_to_take)/len(date_ranges))*100}%")


Checking jobs and pay:
2019-02-20 ====> 2019-02-24
Pay: $X
2019-01-17 ====> 2019-01-21
Pay: $X
2019-01-06 ====> 2019-01-16
Pay: $X
Total Pay: $3X
Total Jobs Taken: 3
Total Percent of Jobs Taken: 30.0%


This is a better heuristic than the previous one, but there might be cases where accepting the shortest job might block us from accepting two other jobs. The potential loss is less than the previous idea but it can readily limit us to half the optimal payoff.
